# Lecture 5

In this lecture we’ll review some core Python patterns (loops and basic function patterns), then build up to a few “mind-bending” but extremely useful ideas:

- treating **functions as data** (passing/returning functions)
- using **`map`, `filter`, `reduce`**, and **comprehensions**
- understanding **iterators**, **generators**, and **lazy evaluation**
- using `*args` / `**kwargs` to **construct function calls**
- a final example: writing a small tool that **“vectorizes”** a function

The goal is not to memorize syntax—it’s to recognize *patterns* you can reuse.


### Learning goals

By the end of this notebook you should be able to:

- write loops that iterate cleanly over sequences (and know when you need indices)
- recognize common “input → output” function patterns (number→list, list→number, list→list)
- explain what it means to pass a function as an argument and return a function
- describe (in words) what `map`, `filter`, and `reduce` do, and why they are often *lazy* in Python 3
- explain the difference between a **list** and an **iterator/generator**
- use `*` and `**` to unpack arguments when calling a function


## Loops Patterns


Python `for`-loops are designed to iterate directly over **items**, not over indices.

That means the most common (and most readable) pattern is:

```python
for item in some_list:
    ...
```

You *can* loop over indices (e.g., `for i in range(len(lst)):`), but in Python you usually only do that when you truly need `i`.


In [ ]:
for index in range(10):
    print(index)

#### A note about `range`

In **Python 3**, `range(10)` does **not** build a list in memory. It creates a lightweight *range object* that generates the numbers on demand.

- You can loop over it directly (as above).
- If you want to *see* the values all at once, wrap it with `list(...)`:


In [ ]:
list(range(10))

### Iterating over a list

Most of the time you should iterate over the **items** directly:

```python
for item in lst:
    ...
```

If you come from a “C-style” background, you may be used to writing:

```python
for i in range(len(lst)):
    item = lst[i]
```

That works in Python too, but it’s usually more verbose and easier to get wrong. Prefer the direct style unless you truly need the index.


In [ ]:
lst=['a','b','c']

# "Python style"
for item in lst:
    print(item)
    
# "C style"
for index in range(len(lst)):
    print(lst[index])

### When you need the index: `enumerate`

If you need both the **index** and the **item**, use `enumerate`:

- it produces pairs like `(index, value)`
- you can unpack those pairs directly in the loop header


In [ ]:
list(enumerate(lst))

In [ ]:
for index,item in enumerate(lst):
    print(index,item)

### Iterating over multiple lists: `zip`

If you want to walk through multiple lists *in parallel*, use `zip`.

`zip(lst1, lst2)` pairs up items:

- first with first
- second with second
- and so on…

By default, `zip` stops when the **shortest** input runs out.


In [ ]:
lst1=['a','b','c']
lst2=['A','B','C']

for item1,item2 in zip(lst1,lst2):
    print(item1,item2)


**Important:** in Python 3, many tools like `zip`, `map`, and `filter` return *iterators*.

That means they don’t immediately compute a full list of results. Instead, they produce values **one at a time** as you loop over them (this is called **lazy evaluation**). We’ll come back to why that matters later.


In [ ]:
zip(lst1,lst2)

To make an iterator “do something”, you have to **consume** it by looping over it.

One easy way to force evaluation is to convert it to a list:

```python
list(zip(lst1, lst2))
```

(Just remember: building a list stores *all* results in memory. Iterators are useful when you don’t want to store everything at once.)


In [ ]:
list(zip(lst1,lst2))

In [ ]:
list(zip(range(4),range(10)))

In [ ]:
list(zip("Hello","World"))

## Basic Function Input/Output Patterns

A huge amount of programming comes down to a few repeatable patterns. Here are four you’ll see constantly:

1. **Number → List**: build up a list and return it  
2. **List → Number**: accumulate a single value (count, sum, max, …)  
3. **List → List (same length)**: transform each element (e.g., square every number)  
4. **List → List (shorter)**: filter down to a subset (e.g., only odds)

In each case, the structure is similar:

- create an output container (or accumulator)
- loop
- update the container
- `return` the result


### Number → List

Example: return a list of **odd numbers below** `max_odd` (i.e., we generate candidates and keep the ones that match a condition).


In [ ]:
def odds(max_odd):
    out_list=list()
    
    # Body
    for num in range(max_odd):
        if num%2==1:
            out_list.append(num)
    
    return out_list
        

In [ ]:
odds(13)

### List to Number

In [ ]:
def count_odds(lst):
    my_count=0
    
    # Body
    for num in lst:
        if num%2==1:
            my_count+=1  # my_count = my_count + 1
    
    return my_count

In [ ]:
count_odds([1,2,4,6,7,9])

### List to Same Length List

In [ ]:
def square(lst):
    out_list = list()
    
    for num in lst:
        out_list.append(num*num)  
            
    return out_list

In [ ]:
square([1,2,4,6,7,9])

### List to Shorter List

In [ ]:
def filter_odds(lst):
    out_list = list()
    
    for num in lst:
        if num%2==1:
            out_list.append(num)  
            
    return out_list

In [ ]:
filter_odds([1,2,4,6,7,9])

## An Example...

Functions that input/output lists:

In [ ]:
def multiply_scalar_list(scalar,b):
    out = list()
    for item in b:
        out.append(scalar*item)
    return out

In [ ]:
print(multiply_scalar_list(5,[1,2,3]))

In [ ]:
def multiply_lists(a,b):
    if len(a)!=len(b):
        print("Only can multiply lists of same length.")
        return None
    else:
        out = list()
        for item1,item2 in zip(a,b):
            out.append(item1*item2)
        return out 

In [ ]:
print(multiply_lists([1,2,3],[2,3,4]))
print(multiply_lists([1,2,3],[2,3,4,5]))

We can combine the two functions and generalize: 

In [ ]:
def multiply(a,b):
    if isinstance(a,(float,int)) and isinstance(b,(float,int)):
        return a*b
    elif isinstance(a,list) and isinstance(b,list):
        return multiply_lists(a,b)
    elif isinstance(a,list) and isinstance(b,(float,int)):
        return multiply_scalar_list(b,a)
    elif isinstance(b,list) and isinstance(a,(float,int)):
        return multiply_scalar_list(a,b)
    else:
        print("Invalid input.")
        return None

def multiply_lists(a,b):
    if len(a)!=len(b):
        print("Only can multiply lists of same length.")
        return None
    else:
        out = list()
        for item1,item2 in zip(a,b):
            # Note now we use multiply not * here:
            out.append(multiply(item1,item2))
        return out 
    
def multiply_scalar_list(scalar,b):
    out = list()
    for item in b:
        # Note now we use multiply not * here:
        out.append(multiply(scalar,item))
    return out

Note that the updated versions of `multiply_lists` and `multiply_scalar_list` re-use `multiply`, allowing further generalization.

In [ ]:
print(multiply(2,2))

print(multiply(2,[1,2,3]))
print(multiply([1,2,3],2))

print(multiply([1,2,3],[2,3,4]))

print(multiply([[1,1,1],[2,2,2]], [[3,3,3],[4,4,4]]))


### Functions as Arguments

In [ ]:
def odd(num):
    return num%2==1

def filter_func(lst, func):
    out_list = list()
    
    for num in lst:
        if func(num):
            out_list.append(num)  
            
    return out_list


In [ ]:
filter_func([1,2,4,6,7,9],odd)

In [ ]:
def even(num):
    return num%2==0

filter_func([1,2,4,6,7,9],even)

`filter_func` takes:

- a list `lst`
- a **predicate** function `func` (a function that returns `True`/`False`)

It returns a *new list* containing only the elements of `lst` for which `func(element)` is `True`.


### Functions as Return

In [ ]:
def make_filter(func):

    def filter_func(lst):
        out_list = list()

        for num in lst:
            if func(num):
                out_list.append(num)  

        return out_list

    return filter_func


In [ ]:
filter_odd_0 = make_filter(odd)

In [ ]:
type(filter_odd_0)

In [ ]:
filter_odd_0([1,2,4,6,7,9])

`make_filter` is a **function factory**:

- it takes a predicate function `func`
- it *builds and returns* a new function that filters lists using `func`

This works because the inner function “remembers” (`closes over`) the value of `func` from the outer scope.


## Functions of Functions

In [ ]:
def my_map(f,lst):
    out=list()
    for item in lst:
        out.append(f(item))
    return out

In [ ]:
def square(x):
    return x*x

def cube(x):
    return x*x*x

print(my_map(square,[1,2,3]))
print(my_map(cube,[1,2,3]))

In [ ]:
def operator(f):
    def my_map(lst):
        out=list()
        for item in lst:
            out.append(f(item))
        return out
    return my_map

In [ ]:
square_operator=operator(square)
cube_operator=operator(cube)

print(square_operator([1,2,3]))
print(cube_operator([1,2,3]))

## Lambda functions

Sometimes you need a tiny “one-off” function and it feels silly to write a full `def`.

A `lambda` creates a function *without giving it a name*:

```python
lambda x: x * x
```

A few notes:

- a `lambda` can take any number of arguments, but it must be **a single expression**
- the expression’s value is automatically returned
- if your logic needs multiple lines, `if`/`for` statements, or good readability, use `def` instead


In [ ]:
square = lambda x: x * x

In [ ]:
square(8)

In [ ]:
def square(x):
    return x * x

To appreciate the power of `lambda`, let’s introduce a few built-in “functional programming” tools in Python:

- `map` (transform each element)
- `filter` (keep only elements that pass a test)
- `reduce` (combine a sequence into a single value)

We’ll also compare these to list comprehensions, which are often more readable in Python.


### map

`map(function, iterable)` applies `function` to each element of `iterable`.

In Python 3, `map(...)` returns an **iterator**, so you usually wrap it with `list(...)` when you want to see all results at once.


In [ ]:
list1 = [1,2,3,4,5,6,7,8,9]

In [ ]:
eg = my_map(lambda x:x+2, list1)
print (eg)

In [ ]:
eg = map(lambda x:x+2, list1)
print (eg)

In [ ]:
list(eg)

In [ ]:
def add_two(x):
    return x + 2

eg_0 = map(add_two, list1)
eg_0

In [ ]:
# To see all results at once, convert to a list:
list(map(add_two, list1))

Because `map` is **lazy**, it only computes values when you *consume* the iterator (for example, by looping).

Also note: `map(...)` returns an **iterator**, which means it can be consumed only once. If you need the results multiple times, convert to a list or create a new `map` object.


In [ ]:
eg_0 = map(add_two, list1)

for x in eg_0:
    print(x)

If you want to compute the result, just force it in the following way:

In [ ]:
eg = list(map(lambda x:x+2, list1))
print (eg)

You can also add two lists.

In [ ]:
list2 = [9,8,7,6,5,4,3,2,1]

In [ ]:
eg2 = list(map(lambda x,y:x+y, list1,list2))
print (eg2)

You can use `map` with `lambda`, but you can also pass any regular function (including built-ins like `str`).

In [ ]:
eg3 = list(map(str,eg2))
print (eg3)

In [ ]:
eg2 = list(map(lambda x,y:(x,y), list1,list2))
print (eg2)

### filter

### `filter`

`filter(predicate, iterable)` keeps only the items for which `predicate(item)` is `True`.

In Python 3, `filter(...)` returns an **iterator** (not a list), so you often write:

```python
list(filter(...))
```

to see the results.


In [ ]:
list1 = [1,2,3,4,5,6,7,8,9]

To get the elements which are less than 5,

In [ ]:
list(filter(lambda x:x<5,list1))

Notice what happens when `map()` is used.

In [ ]:
list(map(lambda x:x<5, list1))

If the function you give to `map` returns `True`/`False`, then `map` produces a list of booleans.

`filter`, on the other hand, uses those booleans to decide which original elements to keep.


In [ ]:
list(filter(lambda x:x%4==0,list1))

### `reduce`

`reduce(function, iterable)` repeatedly combines items to produce a single result.

Conceptually, it does something like:

- combine the first two items → get an intermediate result  
- combine that result with the next item  
- repeat until the iterable is exhausted

`reduce` lives in `functools`, so you need to import it.


In [ ]:
from functools import reduce

In [ ]:
reduce(lambda x,y: x+y,[1,2,3])

In [ ]:
reduce(lambda x, y: (x,y), [1, 2, 3, 4, 5])

## Shortcuts

Python has a few compact syntactic forms that can make code shorter. Use them when they improve readability (not just because they’re short).


In [ ]:
if True:
    "True"
else:
    "False"

In [ ]:
"True" if True else "False"

In [ ]:
"True" if False else "False"

In [ ]:
y = 15
x = 5 if y==15 else 13
print(x)

In [ ]:
print("True") if True else print("False")

In [ ]:
x = print("True") if True else print("False")
type(x)

### List Comprehensions

As we have seen above, there is a common pattern where a function takes a list and returns another list of the same size. For example consider:

In [ ]:
out = list()
for i in range(10):
    out.append(i)
out

We can do the same thing in a single line of code using list comprehensions:

In [ ]:
out = [i*i for i in range(10)]
out

In [ ]:
fruits = ["apple", "banana", "cherry", "kiwi", "mango"]
[x for x in fruits if "a" in x]

In [ ]:
list(filter(lambda x: "a" in x, fruits))

### Dictionary Comprehensions

Using a similar syntax, we can quickly build dictionaries:

In [ ]:
{i : chr(65+i) for i in range(4)}

In [ ]:
[(i, chr(65+i)) for i in range(4)]

In [ ]:
dict([(i, chr(65+i)) for i in range(4)])

## Iterators, generators, and lazy evaluation

An **iterator** is an object you can repeatedly call `next(...)` on to get values *one at a time*.

- `iter(some_iterable)` gives you an iterator
- `next(iterator)` gives you the next value
- when the iterator is exhausted, it raises `StopIteration`

A key idea: **iterators are consumed**. Once you loop over them (or convert them to a list), they don’t “reset” automatically.

Consider the following:


In [ ]:
iter_obj=iter([3,4,5,6,7,8,9])
next(iter_obj)

In [ ]:
next(iter_obj)

In [ ]:
list(iter_obj)

In [ ]:
iter_obj = iter([3,4,5,6,7,8,9])

for i in iter_obj:
    print(i)

### Generators and `yield`

A **generator function** looks like a normal function, but it uses `yield` instead of `return`.

- `return` ends the function immediately.
- `yield` *pauses* the function and hands back a value.
- The next time you ask for a value (with `next(...)` or a `for`-loop), the function resumes **right where it left off**, with all its local variables still in memory.

That’s why generators are great when the full result would be large—you can compute values **on demand** instead of building a huge list first.


In [ ]:
def even_list(x):
    out = list()
    while(x!=0):
        if x%2==0:
            out.append(x)
        x-=1
    return out

def even_gen(x):
    while(x!=0):
        if x%2==0:
             yield x
        x-=1

In [ ]:
even_list(10)

In [ ]:
even_gen(10)

In [ ]:
g=even_gen(10)
next(g),next(g)

In [ ]:
list(even_gen(10))

In [ ]:
g=even_gen(10)
g2=even_gen(15)

next(g),next(g2)

In [ ]:
next(g)

In [ ]:
def prime_gen():
    """Generate prime numbers forever (simple, not optimized).

    This is mainly a demo of *generators that keep state* (the `primes` list)
    across many `yield` calls.
    """
    yield 2

    primes = [2]
    x = 3

    def is_prime(n):
        # Check divisibility by known primes up to sqrt(n)
        for p in primes:
            if p * p > n:
                break
            if n % p == 0:
                return False
        return True

    while True:
        if is_prime(x):
            primes.append(x)
            yield x
        x += 2  # only test odd candidates

In [ ]:
g=prime_gen()
[ next(g) for _ in range(100)]

### Generator comprehensions

A **generator comprehension** looks like a list comprehension, but uses parentheses `(...)` instead of brackets `[...]`.

- `[expr for x in ...]` builds the whole list immediately.
- `(expr for x in ...)` produces values **lazily**, one at a time.

This is a compact way to create a generator.


In [ ]:
gen_squares = (i * i for i in range(5))
gen_squares

In [ ]:
next(gen_squares)

In [ ]:
list(gen_squares)  # consume the rest

In [ ]:
# Once a generator is exhausted, it stays exhausted:
next(gen_squares, "done")

In [ ]:
# Compare: list comprehension builds everything immediately
[i * i for i in range(5)]

In [ ]:
gen_squares2 = (i * i for i in range(5))
type(gen_squares2), type([i * i for i in range(5)])

## Recursive functions

A recursive function calls **itself**. To be correct (and to terminate), it must have two parts:

- a **base case**: a condition where the function stops recursing and returns a result
- a **recursive case**: the function calls itself with a *smaller/simpler* input

When you design a recursive algorithm, make sure you can answer:

- What is the base case?
- What value should be returned in the base case?
- How do the arguments change in the recursive call?
- How are results combined as the recursion “unwinds” back to the original call?

(Also: Python has a recursion limit, so deep recursion can crash—iteration is often safer for large inputs.)


In [ ]:
def factorial(n):
    """Return n! (factorial) for n >= 0."""
    if n < 0:
        raise ValueError("factorial is not defined for negative numbers")
    if n in (0, 1):
        return 1
    return n * factorial(n - 1)

In [ ]:
factorial(10)

In [ ]:
def factorial_trace(n):
    """A version that shows the nested structure of the recursive calls."""
    if n < 0:
        raise ValueError("factorial is not defined for negative numbers")
    if n in (0, 1):
        return 1
    return (n, factorial_trace(n - 1))

In [ ]:
factorial_trace(10)

In [ ]:
def recur_fibo(n):
   if n <= 1:
       return n
   else:
       return(recur_fibo(n-1) + recur_fibo(n-2))

In [ ]:
recur_fibo(10)

In [ ]:
[recur_fibo(i) for i in range(10)]

In [ ]:
def rec_range(start, stop=None, step=1):
    """Recursive version of `range` (supports positive `step`)."""
    if stop is None:
        start, stop = 0, start

    if step <= 0:
        raise ValueError("This simple version only supports a positive step.")

    if start >= stop:
        return []
    return [start] + rec_range(start + step, stop, step)

## Constructing Function Arguments

Imagine that you have a function that takes two arguments:

In [ ]:
def f(one,two):
    print(one,two)

If you are in a situation where you have a list where the arguments are stored, you could call the function in this way:

In [ ]:
x=[1,2]
f(x[0],x[1])

A better way is to **unpack** the list into positional arguments using `*`:

In [ ]:
f(*x)

We can see what `*` does with the following example:

In [ ]:
x=[1,2,3]
print(x)
print(*x)


You can do a similar thing with dictionaries:

In [ ]:
y={"one":1,"two":2}
print(*y)
f(*y)

That isn’t quite right: iterating over a dictionary produces its **keys**.

If you want to pass the dictionary as **keyword arguments**, use `**` (and the keys must match the function’s parameter names):

In [ ]:
f(**y)

Note that the expectation here is that the keys match the name of the arguments of the function. So the following doesn't work:

In [ ]:
y={"a":1,"b":2}
f(**y)

## Coding example

Let’s show off the power of Python with an example. In introductory physics you learn the 1‑D kinematics equations for constant acceleration:

- $x = x_0 + v_0 t + \tfrac{1}{2} a t^2$
- $v = v_0 + a t$

We can implement these equations directly as Python functions.


In [ ]:
def x_a_t(a,t,x_0=0.,v_0=0.):
    x = x_0 + v_0 * t + 0.5 * a * t**2
    return x

In [ ]:
def v_a_t(a,t,v_0=0.):
    v=v_0+a*t
    return v

So for example, the position and velocity of a rock dropped from 10 meters after 1 second is simply:

In [ ]:
x_a_t(-9.8,1.,x_0=10.,v_0=0.)

In [ ]:
v_a_t(-9.8,1.)

In physics, 2‑D and 3‑D motion can often be treated as multiple independent 1‑D problems (one per coordinate).

Instead of rewriting the equations three times, we can write a **higher‑order function** that turns a scalar function into a “vectorized” function.

Assume vectors are stored as lists like `[x, y, z]`. Our vectorizer will:

1. take a scalar function $f_0$
2. create a new function that accepts the *same* arguments as $f_0$, but allows some arguments to be **lists**
3. “broadcast” any scalar arguments by repeating them to match the list length (for example, `t` might be a single time used for all coordinates)
4. call $f_0$ element‑by‑element and collect the results into an output list
5. return the new vectorized function


Let’s take this step by step.

First we need a way to determine the “vector length” we’re working with. We’ll look at all list‑valued arguments and find the maximum length:


In [ ]:
args= [[1,2],[1,2,3],[1,2,3,4], 1]

max_len=0
for a in args:
    if isinstance(a,list):
        max_len=max(max_len,len(a))
    
print(max_len)


Here is a more compact way of doing the same thing using `filter` and `map`:

In [ ]:
max_len = max(map(len,
                  filter(lambda x: isinstance(x,list),
                   args)))
print(max_len)

Next, we'll have to check that every argument is of the same length, and make lists out of ones that are not lists:

In [ ]:
def create_new_args(args):
    max_len = max(map(len,
                      filter(lambda x: isinstance(x,list),
                       args)))
    new_args=list()

    for a in args:
        if not isinstance(a,list):
            a0=[a]*max_len
        elif len(a)!=max_len:
            print("Error: all list arguments must have same length.")
            return
        else:
            a0=a
        new_args.append(a0)

    return new_args

Let’s test:

In [ ]:
create_new_args([[1,2],[1,2,3],1])

In [ ]:
create_new_args([[1,2],[3,4],5])

### Quick Quiz

Can you rewrite `create_new_args` as a two lines of code using functional programming, list comprehensions, and shortcuts? How about a single line?

In [ ]:
def create_new_args_0(args):
    max_len = max(map(len,
                      filter(lambda x: isinstance(x,list),
                        args)))

    # Rewrite this section:
    new_args=list()

    for a in args:
        if not isinstance(a,list):
            a0=[a]*max_len
        elif len(a)!=max_len:
            print("Error: all list arguments must have same length.")
            return
        else:
            a0=a
        new_args.append(a0)

    return new_args

In [ ]:
create_new_args_0([[1,2],[3,4],5])

In [ ]:
create_new_args_0([[1,2],[3,4,5],5])

### Back to Vectorizing Functions

Finally we have to call a function on each element and store the results in a new list. We can use `zip` to simplify this operation. Here's an example of how `zip` works:

In [ ]:
list(zip( [1,1,1,1], [2,2,2,2]))

So for the output of `create_new_args` example above, it'll do the following, which is what we want:

In [ ]:
list(zip([1, 2], [3, 4], [5, 5]))

But the following won't work:

In [ ]:
list(zip(create_new_args([[1,2],[3,4],5])))

Recall

In [ ]:
create_new_args([[1,2],[3,4],5])

We need to do:

In [ ]:
list(zip(*create_new_args([[1,2],[3,4],5])))

Back to calling a function on each element and store the results in a new list:

In [ ]:
def apply_func(f,args):
    out=list()
    for new_args in zip(*args):
        out.append(f(*new_args))
    return out

Here is a fancier way to do the same thing:

In [ ]:
def apply_func(f,args):
    return list(map(lambda x: f(*x),zip(*args)))

So putting it all together, here is the (x,y) location of an object dropped from (10,10) after 1 second:

In [ ]:
apply_func(x_a_t,create_new_args([[-9.8,0],1,[10,10]]))

We are not quite done yet… let’s pull all of this into a reusable function factory called `vectorize`:

In [ ]:
def vectorize(f):
    def create_new_args(args):
        max_len = max(map(len,
                          filter(lambda x: isinstance(x,list),
                           args)))
        new_args=list()

        for a in args:
            if not isinstance(a,list):
                a0=[a]*max_len
            elif len(a)!=max_len:
                print("Error: all list arguments must have same length.")
                return
            else:
                a0=a
            new_args.append(a0)

        return new_args
    
    def apply_func(f,args):
        out=list()
        for new_args in zip(*args):
            out.append(f(*new_args))
        return out
    
    def vect_f(*args):
        return apply_func(f,create_new_args(args))
    
    return vect_f

Let's test:

In [ ]:
vect_x_a_t=vectorize(x_a_t)
vect_x_a_t([-9.8,0],1,[10,10])

Or simply:

In [ ]:
vectorize(x_a_t)([-9.8,0],1,[10,10])

Recall the earlier `multiply` example, we can almost recreate it:

In [ ]:
multiply = vectorize(lambda x,y : x*y)

In [ ]:
multiply(2,[1,2,3])

In [ ]:
multiply([3,2,1],[1,2,3])

But not quite.

Why? Because Python’s `*` operator behaves differently depending on types:

- `3 * 4` is numeric multiplication
- `3 * [1, 2]` repeats the list (`[1, 2, 1, 2, 1, 2]`), which is **not** element‑wise scaling

To truly handle nested lists (matrices/tensors) element‑wise, we need recursion (like the earlier `multiply` example), or a library like NumPy.


In [ ]:
multiply(3,[[3,2,1],[1,2,3]])

## Summary

Key takeaways from this lecture:

- **Looping:** iterate over *items* directly; use `enumerate` when you need indices, and `zip` when you need to iterate over multiple sequences together.
- **Common function patterns:** build and return a list; accumulate into a single value; transform a list; filter a list.
- **Functions are values:** you can pass functions as arguments and return functions (this is the foundation of `map`/`filter` and many powerful abstractions).
- **`lambda`:** convenient for small, one‑line functions; use `def` for anything more complex.
- **Functional tools:** `map`, `filter`, and `reduce` can express common patterns concisely—but in Python 3 they often return **iterators**, so they’re evaluated lazily.
- **Iterators & generators:** iterators produce values on demand; generators use `yield` to pause and resume, keeping local state alive between values.
- **Recursion:** always identify the base case and how the recursive case makes progress toward it.
- **Argument unpacking:** `*args` unpacks positional arguments, `**kwargs` unpacks keyword arguments—useful when writing wrapper/helper functions.
- **Vectorizing functions:** the final example combines these ideas into “code that writes code”—turning a scalar function into a function that works over lists.

### Optional practice
- Rewrite one loop in this notebook using (a) a list comprehension and (b) `map`/`filter`. Compare readability.
- Extend `vectorize` so it can handle **nested lists** (hint: recursion).
- Write a generator that yields the Fibonacci sequence efficiently (without the slow recursive definition).
